# Deploy a Serverless Model Server with Nuclio-KFServing
  --------------------------------------------------------------------

The following notebook demonstrates how to deploy **any pickled model** using **[nuclio](https://github.com/nuclio/nuclio)** + **[KFServing](https://github.com/kubeflow/kfserving)** (a.k.a <b>Nuclio-serving</b>)

#### **notebook how-to's**
* Write and test model serving (KFServing) class in a notebook.
* Deploy the model server as a Nuclio-serving function.
* Invoke and test the serving function.

<a id='top'></a>
#### **steps**
**[define a new function and its dependencies](#define-function)**<br>
**[test the model serving class locally](#test-locally)**<br>
**[deploy our serving class using as a serverless function](#deploy)**<br>
**[test our model server using HTTP request](#test-model-server)**<br>

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio

<a id='define-function'></a>
### **define a new function and its dependencies**

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable


In [3]:
%%nuclio cmd -c
pip install -U -q kfserving
pip install -U -q azure
pip install -U -q mlrun

In [4]:
# %nuclio config spec.build.baseImage = "yjbds/mlrun-files:latest"

In [5]:
import kfserving
import os
import numpy as np
from cloudpickle import load as pload

In [6]:
TARGET_PATH = '/User/mlrun/models'
MODEL_FILE = 'lgb-classifier.pkl'

In [7]:
class ClassifierModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, model = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if not model is None:
            self.classifier = model
            self.ready = True

    def load(self):
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), MODEL_FILE)
        self.classifier = pload(open(model_file, 'rb'))
        self.ready = True

    def predict(self, body):
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [8]:
# nuclio: end-code

______________________________________________

<a id='test-locally'></a>
### **test the model serving class locally**
The class above can be tested locally. Just instantiate the class, `.load()` will load the model to a local dir.

> **Verify there is a `model.bst` file in the model_dir path (generated by the training notebook)**

In [9]:
my_server = ClassifierModel('classifier', model_dir='/User/mlrun/models')
my_server.load()

[I 200122 18:51:48 storage:35] Copying contents of /User/mlrun/models to local


### _data_
Make some classification data using scikit learn's `make_classification`:

In [10]:
from sklearn.datasets import make_classification
n_samples = 10
train_size = 0.7
X, y = make_classification(
    n_samples=n_samples,
    n_features=28, 
    random_state = np.random.RandomState(1))

In [11]:
event = {"instances": X.tolist()}


We can use the `.predict(body)` method to test the model.

In [12]:
my_server.predict(event)

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]

<a id='deploy'></a>
### **deploy our serving class using as a serverless function**

In [13]:
from mlrun import new_model_server, mount_v3io
import requests

In [14]:
fn = new_model_server('generic', 
                      models={'classifier_gen': TARGET_PATH}, 
                      model_class='ClassifierModel').apply(mount_v3io())

In [15]:
fn.spec.no_cache = True

In [ ]:
addr = fn.deploy()

[mlrun] 2020-01-22 18:51:52,843 deploy started


<a id="test-model-server"></a>
### **test our model server using HTTP request**

In [ ]:
import json
import requests

resp = requests.post(addr + '/classifier_gen/predict', json=event)

In [ ]:
resp.__dict__['_content'] 

In [ ]:
json.loads(resp.content)

**[back to top](#top)**